<a href="https://colab.research.google.com/github/ggpal7117/Magnificent-Seven-Analysis-with-Python-and-Tableau/blob/main/Mags7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This will be a continuation of the last notebook. A second notebook was created for simplicity, and so it is easier to follow

In [1]:
!pip install stockdex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 35.3 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.13.5
    Uninstalling beautifulsoup4-4.13.5:
      Successfully uninstalled beautifulsoup4-4.13.5
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: curl-c

In [2]:
# # -- Import Libraries
import stockdex
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from stockdex.ticker import Ticker
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import random
from typing import List, Optional
import logging
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [3]:
# -- List of neccesary tickers
MAG_7_TICKERS = ["GOOG", "AMZN", "AAPL", "META", "MSFT", "NVDA", "TSLA"]

In [4]:
# -- Get all other tickers
# Get Sp500 tickers
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

response = requests.get(url, headers=headers)
tables = pd.read_html(response.text)

sp500_tickers_df = tables[0]
sp500_tickers_df["Symbol"] = sp500_tickers_df["Symbol"].str.replace(".", "-")
tickers = sp500_tickers_df.Symbol.tolist()

sp500_universe = [ticker for ticker in tickers if ticker not in MAG_7_TICKERS+["GOOGL"]]

In [9]:
def get_revenue_data_safe(ticker: str):
  """Write a function to properly scrape revenue data from Yahoo Finance for a specific ticker
     Input ticker: String indicates name of ticker
     rtype: Pandas DataFrame
  """
  try:
    t = Ticker(ticker=ticker)
    result = t.yahoo_web_revenue_estimate

    if result is None or result.empty:
      return None

    label_col = result.columns[0]

    # Keep Avg Estimate row
    result = result[result[label_col] == "Avg. Estimate"]
    if result.empty:
      return None

    result = result.melt(
      id_vars=label_col,
      value_vars=[result.columns[1], result.columns[2]],
      var_name="Quarter",
      value_name="Revenue"
    )[["Quarter", "Revenue"]]

    result["Quarter"] = np.where(
      result["Quarter"].str.lower().str.startswith("current"),
      "Current Quarter",
      "Next Quarter"
    )

    result["Company"] = (
      t.full_name.title()
      if ticker in MAG_7_TICKERS
      else "SP500 Avg (Ex-MAG7)"
    )

    result["Ticker"] = ticker
    return result

  except Exception as e:
    # NEVER crash the pipeline
    print(f"[FAIL] {ticker}: {e}")
    return None


In [7]:
def get_revenue_data(ticker: str):
  """Write a function to properly scrape revenue data from Yahoo Finance for a specific ticker
     Input ticker: String indicates name of ticker
     rtype: Pandas DataFrame
  """
  global MAG_7_TICKERS
  global sp500_universe
  t = Ticker(ticker=ticker)

  # Get data
  result = t.yahoo_web_revenue_estimate

  # Subset Data
  result = result.loc[result[result.columns[0]] == "Avg. Estimate"]

  # Properly format data
  result = result.melt(id_vars=result.columns[0], value_vars=[result.columns[1], result.columns[2]], var_name="Quarter", value_name="Revenue")[["Quarter", "Revenue"]]
  result["Quarter"] = np.where(result.Quarter.str.lower().str.strip().str.startswith("current"), "Current Quarter.", "Next Quarter Estimates")

  if ticker in MAG_7_TICKERS:
    result["Company"] = t.full_name.title()
  else:
    result["Company"] = "SP500 Avg (Ex-MAG7)"
  # Return dataframe
  return result

In [10]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def scrape_sp500_revenue_data(
    tickers: List[str],
    delay: float = 0.5,
    batch_size: int = 50,
    max_retries: int = 3
) -> pd.DataFrame:
    """
    Efficiently scrape revenue data for all S&P 500 tickers with error handling.

    Args:
        tickers: List of ticker symbols
        delay: Seconds to wait between requests (0.3-0.5 recommended)
        batch_size: Number of tickers to process before a longer pause
        max_retries: Maximum retry attempts per ticker

    Returns:
        Combined DataFrame with all revenue data
    """
    all_results = []
    failed_tickers = []

    for i, ticker in enumerate(tqdm(tickers, desc="Scraping revenue data")):
        retry_count = 0
        success = False

        while retry_count < max_retries and not success:
            try:
                result = get_revenue_data(ticker)
                all_results.append(result)
                success = True

                # Standard delay between requests
                time.sleep(delay)

                # Longer pause every batch_size tickers to avoid rate limits
                if (i + 1) % batch_size == 0:
                    logger.info(f"Processed {i + 1}/{len(tickers)} tickers. Pausing...")
                    time.sleep(2)

            except Exception as e:
                retry_count += 1
                logger.warning(f"Error scraping {ticker} (attempt {retry_count}/{max_retries}): {str(e)}")

                if retry_count < max_retries:
                    # Exponential backoff
                    wait_time = delay * (2 ** retry_count)
                    time.sleep(wait_time)
                else:
                    failed_tickers.append(ticker)
                    logger.error(f"Failed to scrape {ticker} after {max_retries} attempts")

    # Log summary
    logger.info(f"Successfully scraped: {len(all_results)}/{len(tickers)}")
    if failed_tickers:
        logger.warning(f"Failed tickers ({len(failed_tickers)}): {failed_tickers}")

    # Combine all results
    if all_results:
        combined_df = pd.concat(all_results, ignore_index=True)
        return combined_df
    else:
        return pd.DataFrame()

# Usage example
if __name__ == "__main__":
    # Assuming you have these defined
    all_tickers = MAG_7_TICKERS + sp500_universe

    revenue_df = scrape_sp500_revenue_data(
        tickers=all_tickers,
        delay=0.5,  # Adjust based on testing
        batch_size=50,
        max_retries=3
    )

    # Save intermediate results
    revenue_df.to_csv("sp500_revenue_data.csv", index=False)
    print(f"Scraped data shape: {revenue_df.shape}")

Scraping revenue data:  30%|███       | 150/500 [07:52<20:11,  3.46s/it]WARNING:__main__:Error scraping DLR (attempt 1/3): 'NoneType' object has no attribute 'find'
ERROR:__main__:Failed to scrape DLR after 3 attempts
Scraping revenue data:  41%|████      | 205/500 [10:42<14:10,  2.88s/it]WARNING:__main__:Error scraping FOX (attempt 1/3): 'NoneType' object has no attribute 'find'
ERROR:__main__:Failed to scrape FOX after 3 attempts
Scraping revenue data:  71%|███████   | 356/500 [18:35<07:22,  3.07s/it]WARNING:__main__:Error scraping PSKY (attempt 1/3): 'NoneType' object has no attribute 'find'
ERROR:__main__:Failed to scrape PSKY after 3 attempts
Scraping revenue data:  83%|████████▎ | 415/500 [21:41<04:11,  2.95s/it]WARNING:__main__:Error scraping SOLS (attempt 1/3): 'NoneType' object has no attribute 'find'
ERROR:__main__:Failed to scrape SOLS after 3 attempts
Scraping revenue data:  88%|████████▊ | 439/500 [22:58<03:12,  3.15s/it]WARNING:__main__:Error scraping TPL (attempt 1/3): '

Scraped data shape: (990, 3)


In [28]:
# We will now slightly explore, clean, and properly format revenue df
revenue_df.Revenue.str[-1].unique()

array(['B', 'M'], dtype=object)

In [22]:
# Drop rows with invalid revenue currency type by filtering directly
revenue_df = revenue_df[revenue_df.Revenue.str[-1] != "-"]

In [29]:
# Change dtype
revenue_df["Revenue"] = revenue_df.Revenue.apply(lambda x: float(x[:-1])*1e9 if x[-1] == "B" else float(x[:-1])*1e6)
revenue_df.head()

,Quarter,Revenue,Company
0,Current Quarter.,1.112000e+11,Alphabet-Inc
1,Next Quarter Estimates,1.033000e+11,Alphabet-Inc
2,Current Quarter.,2.110500e+11,Amazon
3,Next Quarter Estimates,1.747700e+11,Amazon
4,Current Quarter.,1.382500e+11,Apple-Inc


In [31]:
# Split
mag_7_df = revenue_df.head(14)

In [32]:
mag_7_df

,Quarter,Revenue,Company
0,Current Quarter.,1.112000e+11,Alphabet-Inc
1,Next Quarter Estimates,1.033000e+11,Alphabet-Inc
2,Current Quarter.,2.110500e+11,Amazon
3,Next Quarter Estimates,1.747700e+11,Amazon
4,Current Quarter.,1.382500e+11,Apple-Inc
5,Next Quarter Estimates,1.045500e+11,Apple-Inc
6,Current Quarter.,5.832000e+10,Meta-Platforms
7,Next Quarter Estimates,5.118000e+10,Meta-Platforms
8,Current Quarter.,8.028000e+10,Microsoft-Corporation
9,Next Quarter Estimates,8.124000e+10,Microsoft-Corporation


In [33]:
# Subset
non_mag = revenue_df[14:]

In [36]:
# Group
non_mag = non_mag.groupby('Quarter', as_index=False)['Revenue'].mean()
non_mag["Company"] = "SP500 Avg (Ex-MAG7)"

In [41]:
# Aggregate/concat
revenue_data = pd.concat([mag_7_df, non_mag])
revenue_data

,Quarter,Revenue,Company
0,Current Quarter.,1.112000e+11,Alphabet-Inc
1,Next Quarter Estimates,1.033000e+11,Alphabet-Inc
2,Current Quarter.,2.110500e+11,Amazon
3,Next Quarter Estimates,1.747700e+11,Amazon
4,Current Quarter.,1.382500e+11,Apple-Inc
5,Next Quarter Estimates,1.045500e+11,Apple-Inc
6,Current Quarter.,5.832000e+10,Meta-Platforms
7,Next Quarter Estimates,5.118000e+10,Meta-Platforms
8,Current Quarter.,8.028000e+10,Microsoft-Corporation
9,Next Quarter Estimates,8.124000e+10,Microsoft-Corporation


In [40]:
revenue_df.to_csv("revenue_data.csv", index=False)

In [136]:
# Get revnue by product and service
def get_detailed_revenue_data(ticker: str):
  """Write a function to properly scrape revenue by products and service"""
  t = Ticker(ticker=ticker)
  result = t.finviz_revenue_by_products_and_services()

  #finviz_data = t.finviz_revenue_by_segment()

  if ticker in ["GOOG", "META", "NVDA", "MSFT"]:
    finviz_data = t.finviz_revenue_by_segment()
  else:
    finviz_data = t.finviz_revenue_by_products_and_services()

  all_product_revenue_dfs = []
  for product, df in finviz_data.items():
      df['product_service'] = product
      all_product_revenue_dfs.append(df)

  # Concatenate all DataFrames into a single one
  finviz_revenue_df = pd.concat(all_product_revenue_dfs, ignore_index=True)

  # filter products
  if ticker != "NVDA": # Nvda has a lot of missing data
    key_prods = finviz_revenue_df.groupby('product_service', as_index=False)["value"].median()
    key_prods = list(key_prods.query('value != 0')["product_service"].values)
  else:
    key_prods = list(finviz_revenue_df.product_service.unique())

  # Sort and format
  finviz_revenue_df = finviz_revenue_df.query('product_service in @key_prods')
  finviz_revenue_df = finviz_revenue_df.query('product_service != "Consolidation, Policy"')
  finviz_revenue_df["fiscal_year"] = pd.to_datetime(finviz_revenue_df["fiscal_year"]).dt.year
  finviz_revenue_df = finviz_revenue_df.sort_values(by=["product_service", "fiscal_year"], ascending=[True, True])

  finviz_revenue_df = finviz_revenue_df.query('value != 0')

  finviz_revenue_df = finviz_revenue_df.set_index('fiscal_year')[["product_service", "value"]]
  finviz_revenue_df["Company"] = t.full_name.title()

  return finviz_revenue_df

In [137]:
detailed_rev = pd.DataFrame()
# Get all needed data
for ticker in tqdm(MAG_7_TICKERS):
  print(f"**========================{ticker}===========================**")
  df = get_detailed_revenue_data(ticker)
  df["Company"] = Ticker(ticker=ticker).full_name.title()
  display(df)

  detailed_rev = pd.concat([detailed_rev, df])

  print(f"Unique Item: {df.product_service.nunique()}")
  print("**====================================================================**")
  print()
  time.sleep(1.5)

  0%|          | 0/7 [00:00<?, ?it/s]

**========================GOOG===========================**


,product_service,value,Company
fiscal_year,,,
2018,Google Cloud,5.838000e+09,Alphabet-Inc
2019,Google Cloud,8.918000e+09,Alphabet-Inc
2020,Google Cloud,1.305900e+10,Alphabet-Inc
2021,Google Cloud,1.920600e+10,Alphabet-Inc
2022,Google Cloud,2.628000e+10,Alphabet-Inc
2023,Google Cloud,3.308800e+10,Alphabet-Inc
2024,Google Cloud,4.322900e+10,Alphabet-Inc
2018,Google Services,1.305240e+11,Alphabet-Inc
2019,Google Services,1.518250e+11,Alphabet-Inc


Unique Item: 3
**====================================================================**



 14%|█▍        | 1/7 [00:06<00:41,  6.88s/it]

...

 86%|████████▌ | 6/7 [00:39<00:06,  6.53s/it]

**========================TSLA===========================**


,product_service,value,Company
fiscal_year,,,
2019,Automotive Leasing,8.690000e+08,Tesla
2020,Automotive Leasing,1.052000e+09,Tesla
2021,Automotive Leasing,1.642000e+09,Tesla
2022,Automotive Leasing,2.476000e+09,Tesla
2023,Automotive Leasing,2.120000e+09,Tesla
2024,Automotive Leasing,1.827000e+09,Tesla
2019,Automotive Regulatory Credits,5.940000e+08,Tesla
2020,Automotive Regulatory Credits,1.580000e+09,Tesla
2021,Automotive Regulatory Credits,1.465000e+09,Tesla


Unique Item: 4
**====================================================================**



100%|██████████| 7/7 [00:46<00:00,  6.63s/it]


In [138]:
detailed_rev

,product_service,value,Company
fiscal_year,,,
2018,Google Cloud,5.838000e+09,Alphabet-Inc
2019,Google Cloud,8.918000e+09,Alphabet-Inc
2020,Google Cloud,1.305900e+10,Alphabet-Inc
2021,Google Cloud,1.920600e+10,Alphabet-Inc
2022,Google Cloud,2.628000e+10,Alphabet-Inc
...,...,...,...
2020,Services And Other,2.306000e+09,Tesla
2021,Services And Other,3.802000e+09,Tesla
2022,Services And Other,6.091000e+09,Tesla


In [140]:
# Save data
detailed_rev.to_csv("detailed_revenue_data.csv")

In [151]:
dd = detailed_rev.groupby("Company")["product_service"].unique()
dd.explode()

,product_service
Company,
Alphabet-Inc,Google Cloud
Alphabet-Inc,Google Services
Alphabet-Inc,Other Segments
Amazon,Advertising
Amazon,Amazon Web Services
Amazon,Online Stores
Amazon,Other Services
Amazon,Physical Stores
Amazon,Subscription Services


In [131]:
t = Ticker(ticker="MSFT")
t.finviz_revenue_by_segment()

{'Intelligent Cloud':   fiscal_year report_end_date  \
 0        2016      2016-06-30   
 1        2017      2017-06-30   
 2        2018      2018-06-30   
 3        2019      2019-06-30   
 4        2020      2020-06-30   
 5        2021      2021-06-30   
 6        2022      2022-06-30   
 7        2023      2023-06-30   
 8        2024      2024-06-30   
 9        2025      2025-06-30   
 
                                    source_filing_url         value  
 0  https://www.sec.gov/Archives/edgar/data/789019...  2.495200e+10  
 1  https://www.sec.gov/Archives/edgar/data/789019...  2.740700e+10  
 2  https://www.sec.gov/Archives/edgar/data/789019...  3.221900e+10  
 3  https://www.sec.gov/Archives/edgar/data/789019...  3.898500e+10  
 4  https://www.sec.gov/Archives/edgar/data/789019...  4.836600e+10  
 5  https://www.sec.gov/Archives/edgar/data/789019...  5.972800e+10  
 6  https://www.sec.gov/Archives/edgar/data/789019...  7.496500e+10  
 7  https://www.sec.gov/Archives/edgar/data